# Step 3: Record selection

In this step, the records for the study are selected. The previous file is loaded with:

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('demo-merged.csv')

In [4]:
from techMiner import nan2none
df = nan2none(df)

## Group management

Record selection is based in the presence or ausence of terms in each row of a dataframe.

To mark the registers, two new columns called `'INCLUDE'` and `'EXCLUDE'` will be created. Each of them is Boolean. In this tutorial, we call group to this class of columns.  

In the next code, both columns are created with a default value of `None`. 

In [5]:
df['SELECTED'] = None

## Record visualization

**techMiner** implements the `displayRecords()` to visualize a portion of a dataframe in json format.

In [6]:
from techMiner import displayRecords

displayRecords(df[['Title', 'Author Keywords', 'Index Keywords']].head(5))

-----------------------------------------------
Record index: 0
{
  "Author Keywords": null,
  "Index Keywords": "Algorithmic trading; Experimental demonstrations; Fibre-optic communication; Signal transmission; State-of-the-art technology; Transmission bandwidth; Wavelength division multiplexed; Wide bandwidth; Bandwidth; Data communication systems; Fibers; Glass fibers; Light transmission; Light velocity; Optical fibers; Supercomputers; Vacuum; Wave transmission",
  "Title": "Towards high-capacity fibre-optic communications at the speed of light in vacuum"
}
-----------------------------------------------
Record index: 1
{
  "Author Keywords": "High-frequency trading; Limit order markets; Liquidity; Market quality; NASDAQ; Order placement strategies",
  "Index Keywords": null,
  "Title": "Low-latency trading"
}
-----------------------------------------------
Record index: 2
{
  "Author Keywords": null,
  "Index Keywords": null,
  "Title": "Rise of the machines: Algorithmic trading in

## Keywords completation

This step aims to create a column (field) in the dataframe containing key terms for document selection. First, the columns `'Author Keywords'` and `'Index Keywords'` are joined using the `'merge_fields'` function. The new column is called `'keywords'`.

In [7]:
from techMiner import merge_fields
merge_fields(df['Author Keywords'], df['Index Keywords'], sepA=';', sepB=';')[0:10]

0    Algorithmic trading;Experimental demonstration...
1    High-frequency trading;Limit order markets;Liq...
2                                                 None
3    competition (economics);financial market;marke...
4    High-frequency trading;Market making;Market qu...
5    Financial disclosure;Individual characteristic...
6                                                 None
7    Profitability;Automated social engineering;Soc...
8                                                 None
9                                                 None
dtype: object

In [8]:
df['keywords'] = merge_fields(df['Author Keywords'], df['Index Keywords'], sepA=';', sepB=';')

However, there are 51 records without `'Author Keywords'` and `'Index Keywords'`.

In [9]:
len(df[df['keywords'].map(lambda x: x is None)])

51

In [10]:
from techMiner.keywords import Keywords
kyw = Keywords()
kyw.add_keywords(df['keywords'], sep=';')
kyw._keywords[0:10]

['(location-based) publish/subscribe',
 '10-fold cross-validation',
 '?-stable processes',
 'ABM',
 'ACE',
 'ADALINE',
 'ADCC-GARCH',
 'AI techniques',
 'ANFIS',
 'ANFIS ensemble']

In [11]:
## remove copyright
import numpy as np
df['Abstract'] = df['Abstract'].map(lambda x: x[0:x.find('\u00a9')] if isinstance(x, str) and x.find('\u00a9')!= -1 else x)

In [12]:
title_abstract =  merge_fields(df['Title'], df['Abstract'])

In [13]:
## se extrae del titulo y del abstract
keywords_title_abstract = title_abstract.map(lambda x: kyw.extract(x))

In [14]:
idx = df['keywords'].map(lambda x: x is None)
df.loc[idx, 'keywords'] = keywords_title_abstract[idx]

In [15]:
len(df[df['keywords'].map(lambda x: x is None)])

0

In [ ]:
##
## Eliminación de conferencias

In [34]:
conf = Keywords()
conf.add_keywords(['Conferce', 'Proceeding', 'Workshop'], sep=';')
df['SELECTED'] = df['Title'].map(lambda x: False if conf.find(x) is True else None)

In [35]:
for x in df[df['Conference']]['Title']:
    print(x)

ICSIT 2019 - 10th International Conference on Society and Information Technologies, Proceedings
ACM International Conference Proceeding Series
Proceedings - 23rd IEEE International Conference on High Performance Computing Workshops, HiPCW 2016
Workshop on Logistik und Echtzeit, Echtzeit 2017
4th Workshop on Engineering Applications, WEA 2017
SISY 2015 - IEEE 13th International Symposium on Intelligent Systems and Informatics, Proceedings
Proceedings - 2013 Tools and Methods of Program Analysis, TMPA 2013
International Workshops on Business Information Systems, BIS 2015
7th European Workshop on Probabilistic Graphical Models, PGM 2014
Self-Organizing Systems - 7th IFIP TC 6 International Workshop, IWSOS 2013, Revised Selected Papers
Modeling Decisions for Artificial Intelligence - 8th International Conference, MDAI 2011, Proceedings
Advances in Artificial Intelligence, SBIA 2010 - 20th Brazilian Symposium on Artificial Intelligence, Proceedings


In [36]:
##
## Limpieza de palabras clave
##
from techMiner import Thesaurus
th = Thesaurus()
th.fit(df.keywords, sep=';')
df['keywords (cleaned)'] = df['keywords'].map(lambda x: th.transform(x, sep=';'))

Ciclo de edicion manual

In [37]:
df[df.SELECTED.map(lambda x: x is not False)][['Title', 'keywords (cleaned)', 'Abstract', 'SELECTED']].to_json(
    'demo-records.json', 
    orient='records', 
    lines=True)

In [ ]:
df['keywords'] = merge_fields(df['keywords'], keywords_title_abstract, sepA=';', sepB='|', new_sep=';')

In [ ]:
df['keywords'][0:10]

In [ ]:
len(df[df['keywords'].map(lambda x: x is None)])

In [ ]:
##
## writing to disk for manual review
##

In [ ]:
df[['Title', 'Abstract', 'Author Keywords', 'Index Keywords', 'SELECTED']].to_json(
    'demo-records.json', 
    orient='records', 
    lines=True)

In [ ]:
df0 = pd.read_json(
    'demo-records-edited.json', 
    orient='records',
    lines=True)

## Manual review using external software

It is possible to save the dataframe to disk for manual review using, for example, an external editor. In the following code, the columns `'Title'`, `'Author Keywords'`, `'Index Keywords'`, `'Abstract'`, `'INCLUDE'` and `'EXCLUDE'` are saved in json format.

In [ ]:
df[['Title', 'Author Keywords', 'Index Keywords', 'Abstract', 'SELECTED']].to_json(
    'demo-records.json', 
    orient='records', 
    lines=True)

In the following figure, a partial view of the file is presented. Note that the use of json format allows the user to identify easily each field. 

![screen_capture_4.jpg](screen_capture_4.jpg)

## Record selection based on keywords

A common strategy for record selection is a manual review of all records and change the value of the field `'Selected'` from `'null'` to `'true'` or `'false'`(here, both Boolean constants are writen in json standard and not in the Python language). However, this strategy is overwhelming for the user, even for a moderate number of records. Moreover, this strategy not is scalable when new records are added to the dataframe.

A first strategy consists in build two sets of rules; the first is based on conditions to include records in the final dataset; the second is based on rules for excluding records of the final dataset. Both sets of rules are based in the presence of specific words in the fields of each record. Finally it is possible to obtain a small set of records that must be filtered manually.

In our example, we are interested in documents about algorithmic trading in the context of machine learning. We will use a first strategy based on keywords lists. The steps are the following:

1. A new field composed fr

In our example, we are interested in documents about algorithmic trading in the context of machine learning. Thus, it is possible to say that documents including the term *algorithmic trading* in the title will be selected as default; however, this is not true in all cases: for example, the rule does not apply to the document *Efficient event processing through reconfigurable hardware for algorithmic trading*. Thus, it is necessary to build complex rules to avoid the selection of incorrect documents. 

In other cases, it is possible to discart records based on its title, for example, the document entitled 

In the next code fragment, the column `'selected'` is created and the order of columns in the dataframe is modified. 

In [ ]:
full['selected'] = [False] * len(full)

full = full[['selected', 'Year', 'Title', 'Authors', 'Author(s) ID', 'Source title', 'Volume',
       'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by',
       'DOI', 'Affiliations', 'Authors with affiliations', 'Document Type',
       'Publication Stage', 'Source', 'EID', 'Abstract', 'Author Keywords']]

full = full.sort_values(by=['selected','Year', 'Title'])

## Data exportation for manual review

The dataframe is exported in json format. This file is usually edited by the analyst by applying inclusion and exclusion criteria.

In [ ]:
full.to_json('demo-automatic.json', orient='records', lines=True)

## Filtering of selected records

We recommend to modify a copy of the file in the previous step. Now, the reviewed file is readed.

In [ ]:
records = pd.read_json('demo-manual.json', orient='records', lines=True)

In this fragment of code, a filter is applied for obtaining only the selected records.

In [ ]:
records = records[records.selected == True]

## Removing of duplicated records

Records with the same title are removed. 

In [ ]:
from techMiner import removeDuplicateRecords

records = removeDuplicateRecords(records, fields='Title', matchType='fingerprint')

## Column edition

Since the bibliographical information is stored as a pandas.DataFrame, the user can apply common techniques for selecting, sorting and modifying the dataframe.

Information exported by Scopus need to be modifyed: first, zero citations are coded as NaN, and must be replaced by zero. Second, several lists end with a ';' that must be deleted.

In [ ]:
records['Cited by'] = records['Cited by'].map(lambda x: 0 if np.isnan(x) else x)

In [ ]:
records['Author Keywords'] = records['Author Keywords'].map(lambda x: x[:-1] if isinstance(x, str) and x[-1] == ';' else x)
records['Author(s) ID'] = records['Author(s) ID'].map(lambda x: x[:-1] if isinstance(x, str) and x[-1] == ';' else x)

## Keywords review

In this step, the column `'Author Keywords'` is review and edited. The aim of this step is to unify strings refering to the same concept. 

In [ ]:
## A dictionary of possible matches is created
from techMiner.lists import makeCleanupDict
d = makeCleanupDict(records['Author Keywords'], sep=';')

The dictionary is printed in json format for manual editing.

In [ ]:
import json
print(json.dumps(d, indent=2, sort_keys=True))

Previous output was copyed and edited directly from cell output.

In [ ]:
x = '''
{
  "Algorithmic trading": [
    "Algorithmic Trading",
    "Algorithmic trading",
    "algorithmic trading"
  ],
  "Algorithmic trading systems": [
    "Algorithmic Trading Systems",
    "Algorithmic trading system",
    "Algorithmic trading systems"
  ],
  "Algorithmic trading, trading strategy": [
    "Algorithmic trading strategy",
    "Algorithmic trading, trading strategy"
  ],
  "Artificial intelligence": [
    "Artificial Intelligence",
    "Artificial intelligence"
  ],
  "Artificial neural networks": [
    "Artificial Neural Networks",
    "Artificial neural network",
    "Artificial neural networks"
  ],
  "Automated trading": [
    "Automated Trading",
    "Automated trading",
    "automated trading"
  ],
  "Automated trading system": [
    "Automated trading system",
    "Automated trading systems",
    "automated trading system"
  ],
  "Big data": [
    "Big Data",
    "Big data"
  ],
  "Data mining": [
    "Data Mining",
    "Data mining"
  ],
  "Decision support system": [
    "Decision support system",
    "decision support systems"
  ],
  "Deep learning": [
    "Deep Learning",
    "Deep learning",
    "Deep Neural Networks",
    "Deep neural networks"
  ],
  "Efficient market hypothesis": [
    "Efficient Market Hypothesis",
    "Efficient market hypothesis"
  ],
  "Financial Markets": [
    "Financial Markets",
    "Financial market"
  ],
  "Financial forecasting": [
    "Financial forecasting",
    "financial forecasting"
  ],
  "Genetic algorithm": [
    "Genetic algorithm",
    "Genetic algorithms"
  ],
  "High frequency trading": [
    "High Frequency Trading",
    "High frequency trading",
    "high frequency trading",
    "High-frequency trading",
    "high-frequency trading"
  ],
  "Holonic systems": [
    "Holonic Systems",
    "Holonic systems"
  ],
  "Intelligent agents": [
    "Intelligent Agents",
    "Intelligent agents"
  ],
  "Limit order book": [
    "Limit order book",
    "limit order book"
  ],
  "Machine learning": [
    "Machine Learning",
    "Machine learning",
    "machine learning"
  ],
  "Portfolio optimization": [
    "Portfolio optimization",
    "portfolio optimization"
  ],
  "Random forests": [
    "Random forest",
    "Random forests"
  ],
  "Recurrent neural networks": [
    "Recurrent neural network",
    "Recurrent neural networks"
  ],
  "Sentiment analysis": [
    "Sentiment Analysis",
    "Sentiment analysis"
  ],
  "Stock market": [
    "Stock Markets",
    "Stock market"
  ],
  "Technical analysis": [
    "Technical Analysis",
    "Technical analysis",
    "technical analysis"
  ],
  "Trading strategy": [
    "Trading strategies",
    "Trading strategy"
  ],
  "agent-based modeling": [
    "Agent-based modeling",
    "agent-based model",
    "agent-based modeling"
  ],
  "classification": [
    "Classification",
    "classification"
  ],
  "complex event processing": [
    "Complex event processing",
    "complex event processing"
  ],
  "complexity": [
    "Complexity",
    "complexity"
  ],
  "convolutional neural networks": [
    "Convolutional neural network",
    "Convolutional neural networks",
    "convolutional neural networks"
  ],
  "cryptocurrency": [
    "Cryptocurrency",
    "cryptocurrency"
  ],
  "directional change": [
    "Directional changes",
    "directional change"
  ],
  "e-Finance": [
    "E-finance",
    "e-Finance"
  ],
  "multiagent systems": [
    "Multi-agent systems",
    "MultiAgent Systems",
    "Multiagent systems",
    "multiagent systems"
  ],
  "neural network": [
    "Neural network",
    "Neural networks",
    "neural network"
  ],
  "pattern recognition": [
    "Pattern recognition",
    "pattern recognition"
  ],
  "portfolio management": [
    "Portfolio management",
    "portfolio management"
  ],
  "quantitative finance": [
    "Quantitative finance",
    "quantitative finance"
  ],
  "regularization": [
    "Regularization",
    "regularization"
  ],
  "sample size": [
    "Sample size",
    "sample size"
  ],
  "simulation": [
    "Simulation",
    "simulation"
  ],
  "stock price": [
    "Stock price",
    "stock price"
  ],
  "technical indicators": [
    "Technical Indicators",
    "Technical indicators",
    "technical indicators"
  ],
  "trading": [
    "Trading",
    "trading"
  ]
}
'''

In the next cell, json is converted to Python object.

In [ ]:
d = json.loads(x)

Then, the dictionary is applyed to the list of `Author Keywords` to unify words and phrases.

In [ ]:
from techMiner.lists import applyCleanupDict
records['Keywords'] = records['Author Keywords'].map(lambda x: applyCleanupDict(x, d, sep=';')) 

In [ ]:
records['Keywords']

In [ ]:
records.to_json('demo-keywords.json', orient='records', lines=True)

In the next part of this tutorial, the most basic analyses are exemplifyed.